## Frequency Shift

In [ ]:
%pylab inline
from caf_verilog.freq_shift import FreqShift

In [ ]:
fs = 625e3
freq_res = 200
n_bits = 8

In [ ]:
n = np.arange(0,10e3)
x = np.exp(2*np.pi*((50e3)/fs)*n*1j)

In [ ]:
Px, ff = psd(x,2**10, fs)
plot(ff, 10*np.log10(Px))
xlim([0, 100e3])
ylim([-80, 0])

In [ ]:
fq = FreqShift(x, freq_res, fs, n_bits)

In [ ]:
fq.gen_tb(20e3)

In [ ]:
max(fq.x_quant.real), min(fq.x_quant.real), max(fq.x_quant.imag), min(fq.x_quant.imag)

In [ ]:
max(x.real), min(x.real), max(x.imag), min(x.imag)

In [ ]:
Px, ff = psd(fq.x_quant,2**10, fs)
plot(ff, 10*np.log10(Px))
xlim([0, 100e3])
ylim([-80, 40])

## Generated Shifted Signal

In [ ]:
from caf_verilog.io_helper import read_complex_output
gss = read_complex_output(fq.test_output_filename)
sss = read_complex_output(fq.submodules['sig_gen'].test_output_filename)

In [ ]:
print(len(gss), len(x))

In [ ]:
assert len(gss) == len(x)

In [ ]:
print(gss[:5])

In [ ]:
assert len(gss) < len(sss)
for ii in range(len(gss)):
    check_res = fq.x_quant[ii] * sss[ii]
    check_i = int(check_res.real) >> n_bits
    assert gss[ii].real == check_i
    check_q = int(check_res.imag) >> n_bits
    assert gss[ii].imag == check_q

In [ ]:
Px, ff = psd(sss,2**12, fs)
plot(ff, 10*np.log10(Px))
xlim([15000, 25000])
ylim([-80, 20])

In [ ]:
Px, ff = psd(gss,2**12, fs)
plot(ff, 10*np.log10(Px))
xlim([0, 100e3])
ylim([-80, 40])

## Generated Negative Shifted Signal

In [ ]:
%mkdir -p ./neg_shift

In [ ]:
fqn = FreqShift(x, freq_res, fs, 8, neg_shift=True, output_dir='./neg_shift')

In [ ]:
fqn.gen_tb(20000)

In [ ]:
import os
gssn = read_complex_output(os.path.join(fqn.output_dir, fqn.test_output_filename))

In [ ]:
assert len(gssn) == len(x)

In [ ]:
sssn = []
for ss in sss:
    sssn.append(ss.real + (ss.imag*-1j))

In [ ]:
for ii in range(len(gssn)):
    check_res = fqn.x_quant[ii] * sssn[ii]
    check_i = int(check_res.real) >> 8
    assert gssn[ii].real == check_i
    check_q = int(check_res.imag) >> 8
    assert gssn[ii].imag == check_q

In [ ]:
Px, ff = psd(gssn,2**12, fs)
plot(ff, 10*np.log10(Px))
xlim([0, 100e3])
ylim([-80, 40])